<a href="https://colab.research.google.com/github/stephfz/pyanalytics-webscraping101/blob/master/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AboutMe

**Stephanie Frias** / [@_stephfz_](https://twitter.com/_stephfz_)


*   Ingeniera de Software 👩‍💻
*   Sr. SDET @ Evernote 🐘
*   Peruana 🇵🇪




# Web Scraping con python

¿Qué es webscraping?
Es la extracción de data que podemos encontrar en alguna website





## ¿Cómo funciona un web scraper?
Un web scraper toma una url determinada y "se carga" todo el contenido HTML. Es por esto que súper importante entender a full HTML y conocer de css ya que podremos iniciar la extraccion indicandole al scraper en que etiquetas o nodos encontrará la data que le requerimos. 


# ¿Qué vamos a emplear?

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install requests

# Web Scraping de Webs Estáticas 

## "Cargando" la pagina

In [ ]:
import requests
import pprint

URL = 'https://www.buscalibre.cl/libros/search?q=bailarina'
page = requests.get(URL)
pprint.pprint(page.content)

## Extrayendo un _SOLO_ elemento

In [27]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
#buscamos todos los elementos que esten marcados con las clase producto 
results = soup.find_all('div',class_='producto')
print(len(results))

40


In [ ]:
# tambien podemos todos los resultados
print(soup.prettify())

In [7]:
print(results[1])

<div class="producto dcto30 testgrillameta producto1257131" data-id_producto="1257131" data-precio="1870"><a href="https://www.buscalibre.cl/libro-celestina-la-vaca-bailarina/9789872215927/p/1257131" title=" Celestina la Vaca Bailarina  "><div class="imagen"><img alt="Celestina la Vaca Bailarina - Ana Iriarte - David Donayo" class="lazyload" data-src="https://images.cdn3.buscalibre.com/fit-in/180x180/52/48/524859d2bcd89acd0b2806bf263ce968.jpg"/></div><div class="nombre margin-top-10 text-align-left"> Celestina la Vaca Bailarina  </div><div class="autor">Ana Iriarte</div><div class="box-stars hide-on-hover" style="height:23px;"></div><div class="autor color-dark-gray metas hide-on-hover">
                        David Donayo, 2005, Tapa blanda, Nuevo
                </div><div class="box-precio-v2 row margin-top-10 hide-on-hover"><div class="box-dcto col-xs-5" style=""><div class="descuento-v2 color-white position-relative">25%<div class="font-size-small font-weight-light">dcto</div></d

**Obteniendo un valor por etiqueta dentro de un elemento de la sopa**

In [8]:
element = results[1] 
book_title = element.find('a')['title']
print(book_title)

 Celestina la Vaca Bailarina  


**Obteniendo el Autor**

In [9]:
element.find('div',class_='autor').text

'Ana Iriarte'

**Obteniendo Atributos custom**

In [10]:
print(element['data-precio'])

1870


**Obteniendo data de un elemento anidado**

In [11]:
print(element.find_all('div', class_='descuento-v2')[0].text)

25%dcto


In [13]:
import requests
URL = 'https://www.buscalibre.cl/libros/search?q=bailarina&page=100'
response = requests.get(URL)
not_found = 'Lo sentimos, pero no encontramos lo que buscas'
print(response.status_code)
print (not_found not in response.text)

200
False


**Juntando todo y a obtener los datos de todos los resultados**



In [14]:
titles = []
prices = []
discounts = []
authors  = []

not_found_msg = 'Lo sentimos, pero no encontramos lo que buscas'
URL = 'https://www.buscalibre.cl/libros/search?q=bailarina&page='
pag_index = 1

from bs4 import BeautifulSoup


while (True):
  URL = 'https://www.buscalibre.cl/libros/search?q=bailarina&page={0}'.format(pag_index)
  response = requests.get(URL)
  if (not_found_msg not in response.text):
    soup = BeautifulSoup(response.content, 'html.parser')
    #buscamos todos los elementos que esten marcados con las clase producto 
    results = soup.find_all('div',class_='producto')
    for element in results:
      titles.append(element.find('a')['title'])
      authors.append(element.find('div',class_='autor').text)
      prices.append(element['data-precio'])
      discounts.append(element.find_all('div', class_='descuento-v2')[0].text)
    pag_index+= 1
  else:
    break;    

In [15]:
print(titles)

[' La Bailarina de Auschwitz  ', ' Celestina la Vaca Bailarina  ', ' El Musico y la Bailarina ', ' Bailarinas a la Moda  ', ' La Bailarina de Auschwitz  ', ' La Bailarina de Auschwitz  ', ' Milagros (Bailarinas  ', ' La Bailarina y el Rubi  ', ' Nerina, la Bailarina  ', ' La Bailarina de izu  ', ' Primera Bailarina y Otras Aventuras  ', ' Bailarina de Auschwitz, la  ', ' La Bailarina de izu  ', ' La Bailarina de izu  ', ' La Bailarina; El Clon  ', ' Bailarina  ', ' La Bailarina de Auschwitz  ', ' Princesas y Bailarinas  ', ' La Bailarina; El Clon  ', ' La Bailarina y el Inglés  ', ' Bailarina ', ' Bailarinas  ', ' Bailarina!  Escucha la Musica de Ballet!  ', ' LA BAILARINA DE IZU ', ' La Bailarina de Auschwitz ', ' ¡Splash!  ', ' Bailarinas  ', ' Diario Secreto de las Bailarinas ', ' CENICIENTA  ', ' UNA BAILARINA BARBIE LIBRO PARA COLOREAR ', ' El Pirata y la Bailarina  ', ' El Misterio De La Bailarina  ', ' Bailarina Vueltas y Piruetas Pasatiempos  ', ' Mi Princesa Bailarina  ', ' La

In [16]:
import pandas as pd
data = {'title' : titles, 'author':authors, 'price':prices, 'dscto':discounts}
df = pd.DataFrame(data)
df.tail(30)

,title,author,price,dscto
234,La Ballarina: 8 (Petits Plaers) (libro en Cat...,Ōgai Mori,20690,28%dcto
235,"BARBIE, 12 PRINCESAS BAILARINAS",,38820,
236,El Bailarín Pleinair,Patricio Bulnes,38820,
237,La condesita y su bailarín: comedia en un pró...,"Maura, Honorio",21140,
238,EL SOLDADITO DE PLOMO. EL SOLDAT DE PLOM. DAN...,,29120,
239,Cuento Del Perro Bailarín.,"Trigos, Juan.",87960,
240,Yo Soy Bailarina,"Dick, Savage, Peter, González - Blanco, Esthe ...",45340,
241,Dos bailarines en la pista (Carlos de Foucauld),"Lepetit, Charles",28380,
242,vestuario en pegatinas bailarinas,,18060,28%dcto
243,"anna pawlowa, la bailarina del zar",schmidt aleman henz p.,65230,


In [17]:
# remover el % de la columan dscto
df['dscto'] = df['dscto'].apply(lambda x : x[:x.find('%dcto'):])
df.head()

,title,author,price,dscto
0,La Bailarina de Auschwitz,Edith Eger,13600,15
1,Celestina la Vaca Bailarina,Ana Iriarte,1870,25
2,El Musico y la Bailarina,Ximena Bruno,7990,
3,Bailarinas a la Moda,Latinbooks,2690,25
4,La Bailarina de Auschwitz,Edith Eger,17980,32


# Web Scraping con Webs Dinámicas

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:

import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

driver.get("http://www.segurosimple.com/")

element = driver.find_element_by_id("Marca")
#modelo = "Daewoo"
modelo = "Alfa-Romeo"
element.send_keys(modelo)

annio_fabricacion = "2008"
element = driver.find_element_by_id("Año")
element.send_keys(annio_fabricacion)


element = driver.find_element_by_id("Modelo")
assert len(element.text) > len(":: Seleccione ::"), \
        "Los modelos de la marca %s no fueron cargados" % modelo
modelo = "147 1.6 TI"
element.send_keys(modelo)

driver.find_element_by_id("TelefonoCotizacion").send_keys("997261789")

element = driver.find_element_by_id("EmailCotizacion")
element.send_keys("mimail@yahoo.com")

element = driver.find_element_by_id("edadUsuario")
element.send_keys("30")


driver.find_element_by_id("btnComparar").click()

# Mala practica 
time.sleep(3)
print(driver.page_source)  # results

In [40]:
from bs4 import BeautifulSoup
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [46]:
container = soup.find("div",{"id":'CompadadorResultadoDesktop'}) 
rows = container.findAll('tr')
for row in rows:
  broker_img, broker_price = None, None
  broker_img = (row.find_all('td')[0]).find('img')
  if broker_img != None:
    broker_name = (broker_img['alt'])
    broker_price = (row.find_all('td')[1]).find('h3')
    if broker_price != None:
      broker_price = (broker_price.text)
    print (broker_name,broker_price)    


Mapfre $426
La Positiva $429
Rimac $518


# Recursos Útiles
* [Guía de inicio de Beautiful Soup](https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/)
* [Selectores CSS](https://flukeout.github.io/)
* [Primeros para pasos con Selenium, ejemplos con python](https://realpython.com/modern-web-automation-with-python-and-selenium/://)